<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<br><br>
This assignment requires three (3) key steps:
<ol>
    <li>Create a Toronto dataframe with 3 columns: PostalCode, Borough, and Neighborhood.</li>
    <li>Append geographical coordinates to the Toronto dataframe. The resulting dataframe consists of PostalCode, Borough, Neighborhood, Latitude, and Longitude.</li>
    <li>Explore and cluster the neighborhoods in Toronto.</li>
</ol>

<h3>Step 1: Create a Toronto Dataframe (PostalCode, Borough, and Neighborhood)</h3>

In [2]:
# Install beautifulsoup
!pip install bs4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re


# Download the webpage ('List of postal codes of Canada: M' from Wikipedia). Save the text response as variable (html_data)
page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
html_data = requests.get(page_url).text

# Parse the html data
beautiful_soup = BeautifulSoup(html_data,"html5lib")

# Inspect structure
print(beautiful_soup.find("tbody").prettify())

<tbody>
 <tr>
  <td style="width:11%; vertical-align:top; color:#ccc;">
   <p>
    <b>
     M1A
    </b>
    <br/>
    <span style="font-size:85%;">
     <i>
      Not assigned
     </i>
    </span>
   </p>
  </td>
  <td style="width:11%; vertical-align:top; color:#ccc;">
   <p>
    <b>
     M2A
    </b>
    <br/>
    <span style="font-size:85%;">
     <i>
      Not assigned
     </i>
    </span>
   </p>
  </td>
  <td style="width:11%; vertical-align:top;">
   <p>
    <b>
     M3A
    </b>
    <br/>
    <span style="font-size:85%;">
     <a href="/wiki/North_York" title="North York">
      North York
     </a>
     <br/>
     (
     <a href="/wiki/Parkwoods" title="Parkwoods">
      Parkwoods
     </a>
     )
    </span>
   </p>
  </td>
  <td style="width:11%; vertical-align:top;">
   <p>
    <b>
     M4A
    </b>
    <br/>
    <span style="font-size:85%;">
     <a href="/wiki/North_York" title="North York">
      North York
     </a>
     <br/>
     (
     <a href="/wiki/Victoria_Vill

In [3]:
# Data Wrangling (V1): Based on sample provided
table_contents=[]

# Data are within <td> tag
for cell in beautiful_soup.find("table").find_all("td"):
    cell_contents = {}

    # Separate data into PostalCode, Borough, and Neighborhood. Ignore "Not assigned"
    if cell.span.text=="Not assigned":
        pass
    else:
        # PostalCode = Within <p> tag. First 3 characters
        cell_contents['PostalCode'] = cell.p.text[:3]

        # Borough = Within <span> tag. First word(s) before "("
        cell_contents['Borough'] = (cell.span.text).split('(')[0]
        
        # Neighbourhood = Within <span> tag. Text between "(" and ")"
        cell_contents['Neighborhood'] = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
        table_contents.append(cell_contents)

# print(table_contents)

# Extract data into dataframe
toronto_df = pd.DataFrame(table_contents)

# Replace exceptions in specific cells
toronto_df['Borough'] = toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})   
toronto_df              

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [52]:
## Data Wrangling (V2): Cleaner data
## Extract data into dataframe
#toronto_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

#for row in beautiful_soup.find("tbody").find_all("tr"):
#    for cell in row.find_all("td"):
        
#        # Data are within <b> and <span> tags
#        cell_data = cell.find_all(["b", "span"])

#        # Separate data into PostalCode, Borough, and Neighborhood. Ignore "Not assigned"
#        if "Not assigned" not in cell_data[1].text:
#            # PostalCode = Within <b> tag
#            postalcode_text = cell_data[0].text.strip()
            
#            # Neighbourhood = Within <span> tag. Text between "(" and ")"
#            neighborhood_match = re.search("\(.*\)", cell_data[1].text)
#            if neighborhood_match:
#                neighborhood_text = neighborhood_match.group()[1:-1].replace(" /", ",").replace(")", " ").replace("(", " ,").strip()
                
#                # Borough = Within <span> tag. First word(s) before any other tags
#                try: 
#                    borough_text = cell_data[1].contents[0].text
#                except:
#                    borough_text = cell_data[1].contents[0]
                
#                toronto_df = toronto_df.append({"PostalCode": postalcode_text, "Borough": borough_text, "Neighborhood": neighborhood_text}, ignore_index=True)#

#toronto_df              

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
toronto_df.shape

(103, 3)

<h3>Step 2: Append geographical coordinates to the Toronto dataframe (PostalCode, Borough, and Neighborhood, Latitude, Longitude)</h3>

In [8]:
# Geocoder was not reliable. Read from CSV file provided

filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
lat_lng_coords_df = pd.read_csv(filename).rename(columns={'Postal Code': 'PostalCode'})
lat_lng_coords_df

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
# Create new dataframe with geographical coordinates for Toronto
toronto_df = toronto_df.merge(lat_lng_coords_df, how="left")
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h3>Step 3: Explore and cluster the neighborhoods in Toronto.</h3>

In [10]:
# Explore and cluster the neighborhoods in Toronto, where boroughs contain the word Toronto
neighborhood_df = toronto_df[toronto_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
neighborhood_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Download all the dependencies that we will need

In [11]:
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 3.5 MB/s  eta 0:00:01
Libraries imported.


Let's get the geographical coordinates of Toronto.

In [12]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ontario_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<b>Step 3.1:Visualize Toronto and the neighborhoods in it.</b>

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhood_df['Latitude'], neighborhood_df['Longitude'], neighborhood_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'AFLTSZGOIY4WSJSAT0O5KZRL1CPJC5MV2M2YQQCCHM0LFXYY' # your Foursquare ID
CLIENT_SECRET = 'MIMWSRGJSZQP145OPYEENI5AAW3NNLWY4GP5KIJY4F11G5W3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500 # define radius

Get the first neighborhood's latitude and longitude values

In [15]:
neighborhood_latitude = neighborhood_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhood_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhood_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters

In [16]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    RADIUS, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AFLTSZGOIY4WSJSAT0O5KZRL1CPJC5MV2M2YQQCCHM0LFXYY&client_secret=MIMWSRGJSZQP145OPYEENI5AAW3NNLWY4GP5KIJY4F11G5W3&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Send the GET request and examine the results

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60e3ca34aa081c349103d85c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

Borrow the get_category_type function from the Foursquare lab

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


<b>Step 3.2: Explore Neighborhoods in Toronto.</b><br/>
Create a function to repeat the same process to all the neighborhoods in Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [21]:
toronto_venues = getNearbyVenues(names=neighborhood_df['Neighborhood'],
                                   latitudes=neighborhood_df['Latitude'],
                                   longitudes=neighborhood_df['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(1602, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Check how many venues were returned for each neighborhood

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,85,85,85,85,85,85
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36
Davisville North,9,9,9,9,9,9


Find out how many unique categories can be curated from all the returned venues

In [61]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


<b>Step 3.3: Analyze Each Neighborhood</b>

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbor'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbor,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
toronto_onehot.shape

(1602, 234)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [68]:
toronto_grouped = toronto_onehot.groupby('Neighbor').mean().reset_index()
toronto_grouped

,Neighbor,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016949,...,0.00000,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.071429,0.071429,0.142857,0.214286,0.142857,0.000000,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000,0.016129
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,...,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00000,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000


In [72]:
toronto_grouped.shape

(39, 234)

Print each neighborhood along with the top 5 most common venues

In [73]:
num_top_venues = 5

for hood in toronto_grouped['Neighbor']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbor'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1              Bakery  0.05
2        Cocktail Bar  0.05
3      Farmers Market  0.03
4  Italian Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3       Pet Store  0.05
4    Intersection  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2  Airport Terminal  0.14
3           Airport  0.07
4     Boat or Ferry  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.05
2      Sandwich Place  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3      Nightclub  0.06
4    Coff

Put above into a pandas dataframe

In [112]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighbor']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Italian Restaurant,Restaurant,Farmers Market,Seafood Restaurant,Beer Bar,Cheese Shop,Moroccan Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Boat or Ferry,Plane,Rental Car Location,Airport
3,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Salad Place,Ramen Restaurant,Portuguese Restaurant
4,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Restaurant,Candy Store,Athletics & Sports,Baby Store,Nightclub


<b>Step 3.4: Cluster Neighborhoods</b>

In [113]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighbor', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1,
       2, 2, 2, 2, 0, 3, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2], dtype=int32)

In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhood_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Gastropub,Beer Bar,Restaurant,Cocktail Bar,Farmers Market,Italian Restaurant,Bakery,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Bakery,Italian Restaurant,Restaurant,Farmers Market,Seafood Restaurant,Beer Bar,Cheese Shop,Moroccan Restaurant


Visualize the resulting clusters

In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<b>Step 3.5: Examine Clusters</b>

<b>Cluster 1: Playground</b>

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East Toronto,0,Park,Convenience Store,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
33,Downtown Toronto,0,Park,Playground,Trail,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


<b>Cluster 2: Bus Line, Dim Sum Restaurant, Swim School, Department Store, Jewelry Store, Lawyer, Tennis Court, and Comic Shop</b>

In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Bus Line,Dim Sum Restaurant,Swim School,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
20,Central Toronto,1,Gym / Fitness Center,Hotel,Convenience Store,Pizza Place,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Entertainment
21,Central Toronto,1,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
29,Central Toronto,1,Lawyer,Tennis Court,Restaurant,Park,Convenience Store,Comic Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


<b>Cluster 3: Center (with Pubs, Theather, etc)</b>

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,Downtown Toronto,2,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Italian Restaurant
2,Downtown Toronto,2,Café,Coffee Shop,Gastropub,Beer Bar,Restaurant,Cocktail Bar,Farmers Market,Italian Restaurant,Bakery,Japanese Restaurant
3,East Toronto,2,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Bakery,Italian Restaurant,Restaurant,Farmers Market,Seafood Restaurant,Beer Bar,Cheese Shop,Moroccan Restaurant
5,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Salad Place,Ramen Restaurant,Portuguese Restaurant
6,Downtown Toronto,2,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Restaurant,Candy Store,Athletics & Sports,Baby Store,Nightclub
7,Downtown Toronto,2,Coffee Shop,Café,Thai Restaurant,Restaurant,Gym,Hotel,Deli / Bodega,Clothing Store,Cosmetics Shop,Sushi Restaurant
8,West Toronto,2,Bakery,Pharmacy,Café,Recording Studio,Bar,Bank,Supermarket,Middle Eastern Restaurant,Athletics & Sports,Park
10,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Scenic Lookout,Pizza Place,Hotel,Aquarium,Italian Restaurant,Brewery,Sandwich Place


<b>Cluster 4: Health & Beauty Service and Home Service</b>

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Garden,Health & Beauty Service,Home Service,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
